# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd




In [2]:
url= "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.1 Safari/537.36"}
response=requests.get(url,headers=headers)


if response.status_code==200:
    tabla=pd.read_html(response.text)[0]
else:
    print(f"ha ocurrido un error {response.status_code}")

 


/tmp/ipykernel_1580/4285692684.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabla=pd.read_html(response.text)[0]


Eliminamos los valores NaN (en este caso la ultima columna es la que tiene)

In [ ]:
tabla.dropna(axis=1,inplace=True)

print(tabla)

Rank                  object
Song                  object
Artist(s)             object
Streams (billions)    object
Release date          object
dtype: object


Miramos que tipo de dato tienen las columnas, las convertimos en str, para elimnar aquello que no nos interesa y despues las convertimos en el tipo de dato que nos convenga.

In [12]:
print(tabla.dtypes)

Rank                  object
Song                  object
Artist(s)             object
Streams (billions)    object
Release date          object
dtype: object


In [22]:
tabla=tabla.astype(str)
mask = tabla.astype(str).apply(lambda col: col.str.contains(r'\$|B', na=False))
filas_con_simbolos = tabla[mask.any(axis=1)]
print(filas_con_simbolos)


   Rank                                       Song  \
0     1                          "Blinding Lights"   
8     9                                     "Stay"   
10   11                                 "Believer"   
12   13                                   "Lovely"   
16   17                         "I Wanna Be Yours"   
28   29                       "Birds of a Feather"   
32   33                         "Die with a Smile"   
38   39                    "Every Breath You Take"   
39   40                     "Just the Way You Are"   
40   41                        "Bohemian Rhapsody"   
41   42                            "Love Yourself"   
43   44                                  "Shallow"   
48   49                     "Locked out of Heaven"   
49   50                      "When I Was Your Man"   
52   53                           "Mr. Brightside"   
54   55                       "That's What I Like"   
55   56                                  "Bad Guy"   
63   64                     